In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hyperopt import tpe, rand

import datetime

import copy
import json
import time
import os

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.sensitivity import calculate_sensitivity_and_plot
from main.seir.forecast import _order_trials_by_loss, get_forecast, forecast_all_trials, trials_to_df, scale_up_testing_and_forecast
from main.seir.forecast import create_all_csvs, create_decile_csv, create_region_csv, write_csv
from main.seir.uncertainty import MCUncertainty
from utils.create_report import save_dict_and_create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

In [ ]:
forecast_days = 54
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)
ktrials = 10
model=SEIRHD

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, train_period=21, val_period=0, num_evals=1000,
    data_from_tracker=False, initialisation='intermediate', model=model, 
    smooth_jump=True, algo=tpe,
    which_compartments=['active', 'total', 'deceased', 'recovered'])

predictions_dict['m1_rand'] = single_fitting_cycle(
    state, district, train_period=21, val_period=0, num_evals=1000,
    data_from_tracker=False, initialisation='intermediate', model=model, 
    smooth_jump=True, algo=rand,
    which_compartments=['active', 'total', 'deceased', 'recovered'])

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']

In [ ]:
params_array, losses_array = _order_trials_by_loss(predictions_dict['m1'])

for param in params_array[0].keys():
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.hist([param_dict[param] for param_dict in params_array])
    ax.set_title(f'Historgram of parameter {param}')
    ax.set_ylabel('Count')
    plt.show()

In [ ]:
params_array, losses_array = _order_trials_by_loss(predictions_dict['m1_rand'])

for param in params_array[0].keys():
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.hist([param_dict[param] for param_dict in params_array])
    ax.set_title(f'Historgram of parameter {param}')
    ax.set_ylabel('Count')
    plt.show()

In [ ]:
import scipy.stats

In [ ]:
dist = scipy.stats.expon
param = dist.fit([param_dict['E_hosp_ratio'] for param_dict in params_array])

In [ ]:
dist.pdf(np.linspace(1, 1.5, 100), *param[:-2], loc=param[-2], scale=param[-1])

In [ ]:
pdf_fitted = dist.pdf(x, *param[:-2], loc=param[-2], scale=param[-1]) * size